In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
%run train.py --clearml-run-name beta_10 --batch-size 128 --test-batch-size 256 --epochs 101 --lr 0.001 --beta 10.0 --model-name beta_10

ClearML Task: created new task id=f4dae48781234accad9a4b5483b5d93d
2023-09-19 12:35:45,908 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/49f472236d7240f4bd330a30202378ad/experiments/f4dae48781234accad9a4b5483b5d93d/output/log


100%|██████████| 101/101 [16:33<00:00,  9.84s/it]
